In [0]:
from pyspark.sql import DataFrame
from delta.tables import DeltaTable

In [0]:
def check_delta_table(table_name):
    return spark.catalog.tableExists(table_name)

In [0]:
def delta_merge(
    source_table_name: DataFrame
    , target_table_name: str
    , primary_key: str
):
    # people_table.alias("target").merge(
    #     new_df.alias("source"), "target.id = source.id"
    # ).whenNotMatchedInsert(
    #     condition='source._op = "INSERT"',
    #     values={"id": "source.id", "name": "source.name", "age": "source.age"},
    # ).whenMatchedUpdate(
    #     condition='source._op = "UPDATE"',
    #     set={"id": "source.id", "name": "source.name", "age": "source.age"},
    # ).whenMatchedDelete(
    #     condition='source._op = "DELETE"'
    # ).execute()

    target_dt = DeltaTable.forName(spark, target_table_name)
    cols_map = {col : f"source.{col}" for col in source_table_name.columns}
    merge_condition = f"target.{primary_key} = source.{primary_key}"

    target_dt.alias("target").merge(
        source_table_name.alias("source")
        , merge_condition
    ).whenNotMatchedInsert(
        values=cols_map
    ).whenMatchedUpdate(
        set=cols_map
    ).execute()